<a href="https://colab.research.google.com/github/SayyedAliT/Artificial-Intelligence-Course-/blob/main/Project1/Project1_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To develop the SkinLesNet model, the Keras and TensorFlow Python libraries were used

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model


In [2]:
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50


In [3]:
from google.colab import drive


Explain the data Aug








The dataset was split into training and test sets in an 80:20 ratio, with 80% of the data being used for training and 20% being held back for testing,. To ensure that classes were distributed randomly across the sets used for training and testing, the train_test_split function randomly shuffled the data before splitting. This division was compulsory, to assess the model’s generalizability to new data during testing and to guard against overfitting.

In [26]:
dataset_path = '/content/drive/My Drive/AI DataSet/Data-Dermoscopic _Images'

# Initialize the data generators
data_augmentation = ImageDataGenerator(
    preprocessing_function=preprocess_input_vgg16,  # Use appropriate preprocessing for your model
    # rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    # shear_range=0.2,
    # zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Reserve 20% of the data for validation
)



`fill_mode:`

 This parameter specifies how to fill in newly created pixels that can appear after a transformation such as rotation or a width/height shift.

`'nearest':`

 When set to 'nearest', it means that any empty pixels will be filled with the value of the nearest pixel. This is the simplest approach and often works well, avoiding the introduction of artificial colors or patterns into the image.

In [27]:
data_no_augmentation = ImageDataGenerator(
    preprocessing_function=preprocess_input_vgg16,
    validation_split=0.2
)

To achieve uniformity and compatibility with the deep learning model’s input size requirements, it was necessary to standardize the image size. Every image was therefore reduced in size to a square with dimensions of 224 × 224 pixels.

This code sets up the training and validation datasets. It uses the flow_from_directory method to load images from directories, applying the data augmentation defined earlier to the training data. and batches of 32 images are created. class_mode='categorical' indicates that the labels are one-hot encoded.

In [5]:
# Create the training data generator
train_generator = data_augmentation.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Specify this is training data
)

# Create the validation data generator
validation_generator = data_augmentation.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Specify this is validation data
)

Found 140 images belonging to 3 classes.
Found 33 images belonging to 3 classes.


In [16]:
train_generator_no_aug = data_no_augmentation.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator_no_aug = data_no_augmentation.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 140 images belonging to 3 classes.
Found 33 images belonging to 3 classes.


In [21]:
# Load VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Load ResNet50 model
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False


After loading the base models, this section adds custom layers on top. Flatten converts the 2D feature maps into a 1D vector. Dense(3, activation='softmax') adds a fully connected layer with 3 units (one for each class`(carcinoma,melanoma,Nevus)`) and a softmax activation function for multi-class classification.

In [19]:
# For VGG16
x = Flatten()(base_model.output)
predictions = Dense(3, activation='softmax')(x)
model_vgg16 = Model(inputs=base_model.input, outputs=predictions)


This step compiles the model, setting the optimizer, loss function, and metrics to evaluate during training. adam is a popular optimization algorithm, and categorical_crossentropy is a common loss function for multi-class classification tasks.






and next:

the model is trained using the fit method, which takes the training data, validation data, and the number of epochs (iterations over the entire dataset) as arguments.

In [29]:
model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train VGG16
# Train the model
history_with_aug = model_vgg16.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)


Epoch 1/10
5/5 [==============================] - 284s 54s/step - loss: 4.2810 - accuracy: 0.3643 - val_loss: 1.1475 - val_accuracy: 0.2727
Epoch 2/10
5/5 [==============================] - 282s 55s/step - loss: 1.1205 - accuracy: 0.3500 - val_loss: 1.3478 - val_accuracy: 0.3636
Epoch 3/10
1/5 [=====>........................] - ETA: 4:02 - loss: 1.2520 - accuracy: 0.3438

KeyboardInterrupt: 

Finally, the trained model is evaluated on the validation dataset to see how well it performs on unseen data.

In [28]:
vgg16_results = model_without_aug.evaluate(validation_generator_no_aug)

2/2 [==============================] - 19s 480ms/step - loss: 2.4037 - accuracy: 0.3636


In [24]:

x = Flatten()(base_model.output)
predictions = Dense(3, activation='softmax')(x)
model_without_aug = Model(inputs=base_model.input, outputs=predictions)
model_without_aug.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_without_aug = model_without_aug.fit(
    train_generator_no_aug,
    validation_data=validation_generator_no_aug,
    epochs=10
)


Epoch 1/10
5/5 [==============================] - 307s 56s/step - loss: 126.4184 - accuracy: 0.3429 - val_loss: 1.5434 - val_accuracy: 0.3636
Epoch 2/10
5/5 [==============================] - 280s 64s/step - loss: 1.2359 - accuracy: 0.2786 - val_loss: 1.0991 - val_accuracy: 0.3030
Epoch 3/10
5/5 [==============================] - 284s 55s/step - loss: 1.0997 - accuracy: 0.2929 - val_loss: 1.0986 - val_accuracy: 0.3333
Epoch 4/10
5/5 [==============================] - 282s 55s/step - loss: 1.0985 - accuracy: 0.3429 - val_loss: 1.1013 - val_accuracy: 0.0000e+00
Epoch 5/10
5/5 [==============================] - 280s 55s/step - loss: 1.0991 - accuracy: 0.3429 - val_loss: 1.0985 - val_accuracy: 0.3333
Epoch 6/10
5/5 [==============================] - 283s 55s/step - loss: 1.4883 - accuracy: 0.3429 - val_loss: 1.0980 - val_accuracy: 0.3636
Epoch 7/10
5/5 [==============================] - 281s 55s/step - loss: 1.1782 - accuracy: 0.3571 - val_loss: 1.0915 - val_accuracy: 0.3636
Epoch 8/10
5/5

In [ ]:
print("Model with data augmentation:")
print("Training accuracy: ", max(history_with_aug.history['accuracy']))
print("Validation accuracy: ", max(history_with_aug.history['val_accuracy']))

print("\nModel without data augmentation:")
print("Training accuracy: ", max(history_without_aug.history['accuracy']))
print("Validation accuracy: ", max(history_without_aug.history['val_accuracy']))

# ***Part two:***

The performance of the SkinLesNet model was compared to the ResNet50 and VGG16 models. ResNet50  and VGG16  were selected as benchmarks because they are popular CNN architectures known for their effectiveness in image-classification tasks, including medical-imaging applications . ResNet50, part of the ResNet family, utilizes skip connections that aid in mitigating the vanishing gradient problem during training, enabling the network to effectively learn from a broader set of features . On the other hand, VGG16 is recognized for its simple and uniform architecture with multiple convolutional layers, which makes it efficient in learning various image representations . Both models have demonstrated strong performance in image-based tasks, due to their ability to extract meaningful features from medical images, thus making them suitable choices for skin cancer detection